# Analysis Report

Since I feel firearm laws and mass shootings are one of the biggest problems of USA, I decided this will be the topic of my selected data. I found three different datasets (Firearms Provisions in US, US Mass Shootings and NICS firearm background checks), stored as csv format, which I found on the pages Kaggle and BuzzFeedNews github page. 

Extract: First I had to import dependencies, which will allow me to extract, transform and load my data. I imported pandas for extracting the data from csv file to dataframe with read function and for transforming and cleaning it. From SQLAlchemy I imported function “create-engine”, that will connect my notebook to MySQL database and import clean data into MySQL tables. I also imported datetime model, for transforming dates into proper format to analyse data.

Transform: In this phase I meaningly renamed columns for better understanding and to prepare it for MySQL database import, I splitted columns to separate States from Country and Months from Years, I transformed date with datetimes weekday_name function and add another column into Dataframe that help us find the most common day of the week that shootings usually occur, in addition I choose the relevant columns and rename values in order to merge together similar types. One datasets column “years” type had to be transformed from string to integer. Finally I selected the time frame that all datasets have data for (between 1998 and 2017) and I set all indexes to “year”.

Load: I created connection from Pandas to MySQL database with SQLAlchemy "create-engine" function, created database and tables and then checked if the tables exist, next I loaded data to database and confirm data has been added by querying the tables. 

Biggest challange I faced was how to group data to be able to get a meaningful data. I had 3 datasets with different amount of data for certain years. I tried to group it in pandas and import it to MySQL, but was not able to do so and when I grouped and join it in MySQL some data duplicated.If I wouldn't notice that, I would get wrong results when analysing it. I figure the safest way would be to group data by 'State' and 'Year' in pandas, save it to csv file and import that csv file to MySQL. 

Update: I researched what the problem was and how to solve it. Since all the databases have different numbers of rows and different amount of data for a certain state and certain year, I would have to group datasets by year and set an unique id to each year and join databases together based on that id. I would do the same with another table based on state.

In [21]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

### Import CSV into DataFrame

In [22]:
# Mass shootings data
csv_file = "./Resources/Mass Shootings Dataset.csv"
mshooting_data = pd.read_csv(csv_file, encoding='cp1252')
mshooting_data.head()

,S#,Title,Location,Date,Summary,Fatalities,Injured,Total victims,Mental Health Issues,Race,Gender,Latitude,Longitude
0,1,Las Vegas Strip mass shooting,"Las Vegas, NV",10/1/17,NaN,58,515,573,Unclear,NaN,NaN,NaN,NaN
1,2,San Francisco UPS shooting,"San Francisco, CA",6/14/17,"Jimmy Lam, 38, fatally shot three coworkers an...",3,2,5,Yes,Asian,M,NaN,NaN
2,3,Pennsylvania supermarket shooting,"Tunkhannock, PA",6/7/17,"Randy Stair, a 24-year-old worker at Weis groc...",3,0,3,Unclear,White,M,NaN,NaN
3,4,Florida awning manufacturer shooting,"Orlando, Florida",6/5/17,"John Robert Neumann, Jr., 45, a former employe...",5,0,5,Unclear,NaN,M,NaN,NaN
4,5,Rural Ohio nursing home shooting,"Kirkersville, Ohio",5/12/17,"Thomas Hartless, 43, shot and killed a former ...",3,0,3,Yes,White,M,NaN,NaN


In [23]:
# Background checks data
csv_file = "./Resources/nics-firearm-background-checks.csv"
background_data = pd.read_csv(csv_file, encoding='cp1252')
background_data.head()

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,...,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals
0,2019-04,Alabama,26958.0,666.0,6747.0,4538.0,265.0,240,0.0,6.0,...,0.0,0.0,0.0,9.0,2.0,3.0,0.0,0.0,1.0,41146
1,2019-04,Alaska,243.0,10.0,2704.0,2605.0,323.0,215,0.0,1.0,...,0.0,0.0,0.0,16.0,17.0,0.0,0.0,0.0,0.0,6505
2,2019-04,Arizona,6120.0,509.0,12985.0,6750.0,1280.0,731,0.0,10.0,...,1.0,0.0,0.0,19.0,8.0,1.0,0.0,0.0,0.0,30222
3,2019-04,Arkansas,3373.0,843.0,4951.0,4060.0,319.0,303,3.0,10.0,...,0.0,0.0,0.0,8.0,9.0,1.0,0.0,0.0,0.0,15812
4,2019-04,California,46004.0,0.0,37484.0,25895.0,4204.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114466


In [24]:
# Firearms data
csv_file = "./Resources/raw_data.csv"
firearms = pd.read_csv(csv_file, encoding='cp1252')
firearms.head()

,state,year,age18longgunpossess,age18longgunsale,age21handgunpossess,age21handgunsale,age21longgunpossess,age21longgunsale,age21longgunsaled,alcoholism,...,universal,universalh,universalpermit,universalpermith,violent,violenth,violentpartial,waiting,waitingh,lawtotal
0,Alabama,1991,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,15
1,Alaska,1991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,Arizona,1991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,Arkansas,1991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
4,California,1991,0,1,0,1,0,0,0,1,...,1,1,1,1,1,1,1,1,1,58


### Cleaning Data

In [25]:
# MASS SHOOTING DF

# Rename columns to prepare it for MySQL database
mshooting_data = mshooting_data.rename(columns = {"Total victims" : "Total_victims",
                                                  "Mental Health Issues" : "Mental_Health_Issues"})


# Split column to extract a state
mshooting_data['City'], mshooting_data['State'] = mshooting_data['Location'].str.split(',', 1).str

# Split column to extract a Year
mshooting_data['Date'] = pd.to_datetime(mshooting_data.Date)
mshooting_data['Year'] = mshooting_data.Date.dt.year

# Transform date into weekday_name to find the most common day of the week that shootings usually occur
mshooting_data['Weekday'] = mshooting_data.Date.dt.weekday_name

# Choose the relevant columns
mshooting_df = mshooting_data[["Year", "Weekday", "State", "Fatalities", "Injured", "Total_victims", "Mental_Health_Issues", "Race", "Gender"]].copy()

# Rename values in order to merge together similar types
mshooting_df['Race'] = mshooting_df['Race'].replace({
                                     'Other' : 'Unknown',
                                     'Black American or African American' : 'Black',
                                     'White American or European American' : 'White',
                                     'Asian American': 'Asian',
                                     'Two or more races' : 'Unknown',
                                     'Black American or African American' : 'Black',
                                     'White American or European American/Some other Race' : 'White',
                                     'black' : 'Black',
                                     'white' : 'White',
                                     'unclear' : 'Unknown',
                                     'Asian American/Some other race' : 'Asian',
                                     'Native American or Alaska Native' : 'Native American',
                                     'Some other race': 'Unknown'})

mshooting_df['State'] = mshooting_df['State'].replace({
                                     ' NV' : ' Nevada',
                                     ' CA': ' California',
                                     ' PA' : ' Pennsylvania',
                                     ' WA' : ' Washington',
                                     ' D.C.': ' Washington',
                                     ' LA' : ' Louisiana',
                                     ' Lancaster, Pennsylvania': ' Pennsylvania',
                                     ' Boston, Massachusetts': ' Massachusetts',
                                     ' Albuquerque, New Mexico': ' New Mexico',
                                     ' Souderton, Lansdale, Harleysville, Pennsylvania': ' Pennsylvania',
                                     ' San Diego, California': ' California'})

mshooting_df['Gender'] = mshooting_df['Gender'].replace({
                                     'M' : 'Male',
                                     'M/F': 'Unknown',
                                     'Male/Female' : 'Unknown'})

mshooting_df['Mental_Health_Issues'] = mshooting_df['Mental_Health_Issues'].replace({
                                     'unknown' : 'Unknown',
                                     'Unclear': 'Unknown',
                                     'Unclear ' : 'Unknown'})

#Select the years for Analysis
mshooting_df = mshooting_df[(mshooting_df['Year'] >= 1998) & (mshooting_df['Year'] <= 2017)]
mshooting_df.set_index('Year').head()

,Weekday,State,Fatalities,Injured,Total_victims,Mental_Health_Issues,Race,Gender
Year,,,,,,,,
2017,Sunday,Nevada,58,515,573,Unknown,NaN,NaN
2017,Wednesday,California,3,2,5,Yes,Asian,Male
2017,Wednesday,Pennsylvania,3,0,3,Unknown,White,Male
2017,Monday,Florida,5,0,5,Unknown,NaN,Male
2017,Friday,Ohio,3,0,3,Yes,White,Male


In [36]:
# Split column to extract a year
background_data['Year'], background_data['Month'] = background_data['month'].str.split('-', 1).str

# Choose the relevant columns and renaming them
background_df = background_data[["Year", "state", "totals"]].copy()
background_df.rename(columns = {'state': 'State',
                                'totals': 'Total_Checks'}, inplace=True)

# Convert "year" values to integers
background_df["Year"] = background_df["Year"].astype(int)

# Select the years for Analysis
background_df = background_df[(background_df['Year'] >= 1998) & (background_df['Year'] <= 2017)]
background_df.set_index('Year').head()

,State,Total_Checks
Year,,
2017,Alabama,56545
2017,Alaska,8330
2017,Arizona,41559
2017,Arkansas,29559
2017,California,142492


In [27]:
# Choose the relevant columns and renaming them
firearm_laws_df = firearms[["state", "year", "felony", "mentalhealth", "opencarrypermith", "permit", "lawtotal"]].copy()
firearm_laws_df.rename(columns = {'state': 'State',
                                  'year': 'Year',
                                  'felony': 'Felony_Law',
                                  'mentalhealth': 'Mental_Health_Law',
                                  'opencarrypermith': 'Need_permit_to_opencarry',
                                  'permit': 'Need_permit_for_possesion',
                                  'registration': 'Registration_Law',
                                  'training': 'Training_required',
                                  'lawtotal': 'Total_Laws'},inplace = True)

# Select the years for Analysis
firearm_laws_df = firearm_laws_df[(firearm_laws_df['Year'] >= 1998) & (firearm_laws_df['Year'] <= 2017)]
firearm_laws_df.set_index('Year').head()

,State,Felony_Law,Mental_Health_Law,Need_permit_to_opencarry,Need_permit_for_possesion,Total_Laws
Year,,,,,,
1998,Alabama,0,0,0,0,16
1998,Alaska,1,0,0,0,8
1998,Arizona,1,0,0,0,12
1998,Arkansas,1,0,0,0,11
1998,California,1,1,1,0,70


### Create database Connection

In [28]:
# Create connection to a local database
connection_string = "root:sonja@127.0.0.1/shootings_db"
engine = create_engine(f'mysql://{connection_string}')

In [29]:
# Check for tables
engine.table_names()

['background_checks', 'firearm_laws', 'mass_shootings']

### Use pandas to load csv converted DataFrame into database

In [30]:
# Load background_final to MySQL database
background_df.to_sql(name='background_checks', con=engine, if_exists='append', index=False)

In [31]:
# Load mshooting_df to MySQL database
mshooting_df.to_sql(name='mass_shootings', con=engine, if_exists='append', index=False)

In [32]:
# Load firearm_laws_df to MySQL database
firearm_laws_df.to_sql(name='firearm_laws', con=engine, if_exists='append', index=False)

In [33]:
# Confirm data has been added by querying the background_checks table
pd.read_sql_query('select * from background_checks', con=engine).head()

,Year,State,Total_Checks
0,2017,Alabama,56545
1,2017,Alaska,8330
2,2017,Arizona,41559
3,2017,Arkansas,29559
4,2017,California,142492


In [34]:
# Confirm data has been added by querying the mass_shootings table 
pd.read_sql_query('select * from mass_shootings', con=engine).head()

,Year,Weekday,State,Fatalities,Injured,Total_victims,Mental_Health_Issues,Race,Gender
0,2017,Sunday,Nevada,58,515,573,Unknown,None,None
1,2017,Wednesday,California,3,2,5,Yes,Asian,Male
2,2017,Wednesday,Pennsylvania,3,0,3,Unknown,White,Male
3,2017,Monday,Florida,5,0,5,Unknown,None,Male
4,2017,Friday,Ohio,3,0,3,Yes,White,Male


In [35]:
# Confirm data has been added by querying the firearm_laws table
pd.read_sql_query('select * from firearm_laws', con=engine).head()

,State,Year,Felony_Law,Mental_Health_Law,Need_permit_to_opencarry,Need_permit_for_possesion,Total_Laws
0,Alabama,1998,0,0,0,0,16
1,Alaska,1998,1,0,0,0,8
2,Arizona,1998,1,0,0,0,12
3,Arkansas,1998,1,0,0,0,11
4,California,1998,1,1,1,0,70
